<a href="https://colab.research.google.com/github/santhanreddy864/In-class-exercise-1/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [4]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [5]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor


     |████████████████████████████████| 41.4MB 102kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834234 sha256=1621e31ffe073ab7e8fa24ffb2c806d70e582332cbf23b0e113a1f9e3b8a58b5
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [6]:
%%bash
polyglot download sentiment2.en


[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Getting data used Assignment 3**

In [15]:
import pandas as pd
df = pd.read_csv('/content/assign3_reviews.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
from nltk.corpus import stopwords
from textblob import TextBlob
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Review'] = df['Review'].apply(lambda x: str(TextBlob(x).correct()))

In [16]:
from polyglot.text import Text
swords = []
for x in df['Review']:
  text = Text(x)
  for w in text.words:
    if w.polarity != 0:
      swords.append(w)
swords

['amazing',
 'brilliant',
 'magnificent',
 'safe',
 'joke',
 'gritty',
 'insanely',
 'dark',
 'disturbing',
 'terrible',
 'refused',
 'proper',
 'greatest',
 'like',
 'dark',
 'wise',
 'betrayal',
 'joke',
 'better',
 'greatest',
 'like',
 'dark',
 'inevitable',
 'better',
 'joke',
 'joke',
 'fortunate',
 'greatest',
 'top',
 'scars',
 'best',
 'loses',
 'joke',
 'liked',
 'like',
 'fall',
 'problem',
 'hated',
 'masterpiece',
 'worthy',
 'enjoy',
 'slow',
 'joke',
 'crazy',
 'lame',
 'stunned',
 'right',
 'hell',
 'unstable',
 'ironic',
 'joke',
 'accurate',
 'realistic',
 'dark',
 'disturbing',
 'led',
 'worthy',
 'bad',
 'heartbreaking',
 'inevitably',
 'crime',
 'love',
 'like',
 'suffer',
 'better',
 'missed',
 'joke',
 'helped',
 'great',
 'great',
 'great',
 'plot',
 'brilliant',
 'super',
 'great',
 'joke',
 'joke',
 'best',
 'worthy',
 'best',
 'unbelievable',
 'better',
 'win',
 'best',
 'joke',
 'win',
 'best',
 'snub',
 'joke',
 'great',
 'suffering',
 'issues',
 'neglect',

In [23]:
from collections import Counter
fq = Counter(sentimental_words)
rank = pd.DataFrame(list(fq.most_common()), columns=['Word', 'Frequency'])
rank.index = list(range(1, len(fq.most_common())+1))
rank

,Word,Frequency
1,joke,135
2,like,61
3,good,53
4,best,40
5,dark,33
...,...,...
467,cry,1
468,marvelous,1
469,tolerable,1
470,pleasantly,1


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [29]:
def Encoder(senti):
  sent_e = 0
  if senti == 'Positive':
    sent_e = 1
  elif senti == 'Negative':
    sent_e = 2
  return sent_e
a_sent = []
for senti in df['Sentimental Analysis']:
  a_sent.append(Encoder(senti))

from textblob import TextBlob
text_sent = []
for x in df['Review']:
  p = TextBlob(x).sentiment.polarity
  if p > 0:
    text_sent.append(1)
  elif p < 0:
    text_sent.append(2)
  elif p == 0.0:
    text_sent.append(0)
text_blob_df = pd.DataFrame(list(zip(df['Review'], a_sent, text_sent)), columns = ['Review', 'original', 'obtained'])
text_blob_df



,Review,original,obtained
0,1010 amazing story brilliant taking magnificen...,0,1
1,thought movie complete utter garage dont under...,2,1
2,expecting masterpiece oscar worthy film hope a...,2,2
3,joaquin portray sociopathic mentally unstable ...,1,1
4,one best moves years 100 oscar worthy joaquin ...,1,1
...,...,...,...
103,stunning piece work up director told phillips ...,1,1
104,joke basically like edition postmodern version...,2,1
105,people hoped pretensions makes feel like enlig...,2,1
106,movie excellent amazing beginning joaquins per...,1,1


**Vader**

In [30]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [37]:
from sklearn.metrics import f1_score, accuracy_score
text_a = accuracy_score(text_blob_df['original'], text_blob_df['obtained'])*100
text_score = f1_score(text_blob_df['original'], text_blob_df['obtained'], average = 'macro')
print("Text Accuracy = {0} & F1 Score = {1}".format(text_a, text_score))

Text Accuracy = 56.481481481481474 & F1 Score = 0.399400871459695


In [42]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
v_sent = []
for line in df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    v_sent.append(1)
  elif polarity < 0:
    v_sent.append(2)
  elif polarity == 0.0:
    v_sent.append(0)
vader_df = pd.DataFrame(list(zip(df['Review'], a_sent, v_sent)), columns = ['Review', 'original', 'obtained'])
vader_df

,Review,original,obtained
0,1010 amazing story brilliant taking magnificen...,0,1
1,thought movie complete utter garage dont under...,2,2
2,expecting masterpiece oscar worthy film hope a...,2,1
3,joaquin portray sociopathic mentally unstable ...,1,1
4,one best moves years 100 oscar worthy joaquin ...,1,1
...,...,...,...
103,stunning piece work up director told phillips ...,1,1
104,joke basically like edition postmodern version...,2,1
105,people hoped pretensions makes feel like enlig...,2,1
106,movie excellent amazing beginning joaquins per...,1,1


In [45]:
from sklearn.metrics import f1_score, accuracy_score
vader_a = accuracy_score(vader_df['original'], vader_df['obtained'])*100
vader_score = f1_score(vader_df['original'], vader_df['obtained'], average = 'macro')
print("Vader Accuracy = {0} & F1 Score = {1}".format(vader_a, vader_score))

Vader Accuracy = 58.333333333333336 & F1 Score = 0.3673289183222958


**Sentiment Analysis**

In [52]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
T_vector = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)


In [59]:
trainx, testx, trainy, testy = train_test_split(df['Review'],df['Sentimental Analysis'],test_size=0.33)
Encoder = LabelEncoder()
trainy = Encoder.fit_transform(trainy)
testy = Encoder.fit_transform(testy)
trainx_vect = T_vector.fit_transform(trainx)
testx_vect = T_vector.transform(testx)
from sklearn import svm
from sklearn.metrics import classification_report
svm = svm.SVC(kernel='linear')
svm.fit(Train_X_vectors, Train_Y)
predicted = svm.predict(Test_X_vectors)
final = classification_report(Test_Y, predicted, output_dict=True)
final


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.625,
  'precision': 1.0,
  'recall': 0.45454545454545453,
  'support': 11},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 3},
 '2': {'f1-score': 0.8301886792452831,
  'precision': 0.7096774193548387,
  'recall': 1.0,
  'support': 22},
 'accuracy': 0.75,
 'macro avg': {'f1-score': 0.48506289308176104,
  'precision': 0.5698924731182796,
  'recall': 0.48484848484848486,
  'support': 36},
 'weighted avg': {'f1-score': 0.698309748427673,
  'precision': 0.739247311827957,
  'recall': 0.75,
  'support': 36}}

Here by performing above operations and analysing the above results we can see that svm f1 score is greater than the remaining two and vader reamins as the least. so it was better implement svm than compared to vader and Text Blob